# 파이프라인 생성

파이프라인은 yaml 및 tar.gz 파일로 생성되며 kubeflow에 업로드 하면 플로우 차트를 제공합니다.
이러한 파이프라인은 컴포넌트를 통해 구성됩니다. 컴포넌트는 각각 독자적으로 존재하는 컨테이너로 입력 및 출력 값을 통해 흐름을 구성합니다.
각각의 컴포넌트는 사용자의 지시에 따라 구성되며 재사용이 용이합니다.

본 예제에서는 간단한 파이프라인 및 컴포넌트 구성을 확인합니다.

In [ ]:
from typing import NamedTuple

import kfp
from kfp import components, dsl
from kfp.components import func_to_container_op

## 컴포넌트 구성

kubeflow pipeline을 작성함에 있어 먼저 컴포넌트를 생성해야 합니다. <br>
컴포넌트는 함수를 통해 구성이 가능하며 kfp 패키지에서 제공하는 두가지 방법을 예제를 통해 확인합니다.

### func_to_container_op

In [4]:
@func_to_container_op
def print_small_text(text: str):
    '''Print small text'''
    print(text)
    
@func_to_container_op
def small_text():
    return 'test_component'

### create_component_from_func

In [5]:
def print_package_version():
    ''' import package'''
    import tensorflow as tf
    import numpy as np
    
    text = f'tensorflow version : {tf.__version__}'
    print(tf.__version__)
    
    return text
    
print_package_version_op = components.create_component_from_func(
    print_package_version, base_image='tensorflow/tensorflow',
    packages_to_install=['numpy==1.23.2']
)

컴포넌트 생성 시 추가적인 패키지 및 이미지가 필요하다면 위와 같이 설정하여 생성할 수 있습니다.

## 파이프라인 구성

파이프라인은 생성한 컴포넌트의 입력과 출력을 연결하여 생성합니다. 반환 값이 있는 컴포넌트들의 데이터 형태를 정확히 입력해 주어야 합니다.

In [ ]:
@dsl.pipeline(name='pipeline name')
def pipeline():
    '''Pipeline that passes small constant string to to consumer'''
    small_text_task = small_text()
    print_package_version_task = print_package_version_op()
    
    print_small_text1_task = print_small_text(small_text_task.outputs)
    print_small_text2_task = print_small_text(print_package_version_task.outputs)

위와 같이 입력 및 출력을 직접 연결하여 파이프라인을 구성합니다. <br>
컴포넌트의 출력 값은 outputs으로 표현되며 여러 인자가 있다면 직접 작성해야 합니다. 자세한 데이터 전달 방법은 data_passing에서 다룹니다.

In [ ]:
kfp.compiler.Compiler().compile(pipeline, 'pipeline_example.tar.gz')

구성한 파이프라인은 yaml 및 tar.gz 파일로 변환하여 사용한다.
생성된 파일은 kubeflow UI에서 업로드하여 사용이 가능하다.